## Data Integration and Reshaping

environment: pyhton 3.8.8

### Table of Content

### 1 Extrac data:

the xml file is missing root, & should be changed to ''&''

In [19]:
# read the xml file
my_xml = []
with open('realstate.xml') as f:
    for line in f:
        # fix the text issue
        fix_line = line.replace('&','&amp;')
        my_xml.append(fix_line)
    f.close()
# create a new file and add root
with open('fixed_realstate.xml','a') as f:
    f.write("<properties>\n")
    for line in my_xml:
        string = '    ' + line
        f.write(string)
    f.write("</properties>\n")
    f.close()

Parse xml file

In [20]:
import xml.etree.ElementTree as ET
import pandas as pd

xml_data = open('fixed_realstate.xml', 'r').read() 
root = ET.XML(xml_data)
# list for dataframe attributes
property_id  = []
lat  = []
lng =[]
addr_street = []
for i, attribute  in enumerate(root):
    # match the attribute and store information
    for info in attribute:
        if  info.tag == 'property_id':
            property_id.append(info.text)
        elif info.tag == 'lat':
            lat.append(info.text)
        elif info.tag == 'lng':
            lng.append(info.text)
        elif info.tag == 'addr_street':
            addr_street.append(info.text)
        else:
            # in case, If there are any strange errors
            print(info.tag,info.text)
# convert to pandas dataframe
data = [property_id,lat,lng,addr_street]       
address_df = pd.DataFrame(data).T 
cols = ['property_id','lat','lng','addr_street']
address_df.columns = cols

In [21]:
address_df.head()

,property_id,lat,lng,addr_street
0,73532,-37.849075,145.303972,31 Dorrigo Drive
1,2194,-37.819211,144.845848,35 Corrigan Avenue
2,12980,-37.694174,144.758153,32 Dundee Way
3,7687,-37.74916077,144.7638855,95 Fairfax Circuit
4,3210,-37.814997,144.86763100000005,25A Sanderson Street


Read the pdf file

In [22]:
import PyPDF2
# creating a pdf file object
pdfFileObj = open('realstate.pdf', 'rb')
  
# creating a pdf reader object
pdfReader  = PyPDF2.PdfReader(pdfFileObj)

pages_text = []
 
# creating a page object
pageObj = pdfReader.pages[0]
pages_text.append(pageObj.extract_text())

#convert to pandas df
df_pdf = pd.DataFrame({'Text': pages_text})

pdfFileObj.close()

In [45]:
from tabula.io import read_pdf
tables = read_pdf('realstate.pdf', pages='all')

In [46]:
df_pdf = tables[0]
df_pdf = df_pdf.drop('property_id', axis=1)
rename = {'Unnamed: 0': 'property_id'}
df_pdf.rename(columns=rename,
          inplace=True)
df_pdf = df_pdf[['property_id','lat','lng','addr_street']]
df_pdf.head()

,property_id,lat,lng,addr_street
0,34701,-37.722324,145.021654,33 Boldrewood Parade
1,12218,-37.712146,144.771964,12 Lacebark Road
2,95829,-37.925040,145.047606,46 London Street
3,37225,-37.645165,145.052382,3 Guinea Court
4,94289,-37.960549,145.078754,14 Oakmont Crescent


In [96]:
# concat both df
frames = [df_pdf, address_df]
address_df = pd.concat(frames)
address_df.reset_index(inplace=True, drop=True) 

In [97]:
address_df.head()

,property_id,lat,lng,addr_street
0,34701,-37.722324,145.021654,33 Boldrewood Parade
1,12218,-37.712146,144.771964,12 Lacebark Road
2,95829,-37.92504,145.047606,46 London Street
3,37225,-37.645165,145.052382,3 Guinea Court
4,94289,-37.960549,145.078754,14 Oakmont Crescent


### 2.Data Transformation:

In [98]:
address_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2191 entries, 0 to 2190
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   property_id  2191 non-null   object
 1   lat          2191 non-null   object
 2   lng          2191 non-null   object
 3   addr_street  2191 non-null   object
dtypes: object(4)
memory usage: 68.6+ KB


In [104]:
#change the data type of lat and lng to float
address_df['lat'] = address_df['lat'].astype(float)
address_df['lng'] = address_df['lng'] .astype(float)
address_df['property_id'] = address_df['property_id'].astype(int)
address_df['addr_street'] = address_df['addr_street'].astype(str)

In [102]:
# remove duplicated values

address_df.sort_values("property_id", inplace=True)
address_df.drop_duplicates(subset="property_id",
                     keep=False, inplace=True)
address_df.head()

,property_id,lat,lng,addr_street
1120,70,-37.824362,144.962089,117/173 City Road
777,271,-37.807817,144.897262,12 Pole Street
1327,313,-37.806835,144.901581,13 Parker Street
941,402,-37.797932,144.895828,16 Malden Street
1655,403,-37.797905,144.895897,16 Malden Street


In [105]:
# add columns to dataframe and set defualt values
address_df['suburb'] = 'not available'
address_df['closest_train_station_id'] = 0
address_df['distance_to_closest_train_station'] = 0
address_df['travel_min_to_MC'] = -1
address_df['direct_journey_flag'] = -1
address_df['Type'] = 'not identified'


In [ ]:
# read shapefile and extract suburb name and it's boundary coordinat